# Поиск по коллекции (из фактов Википедии)
Шулюгин Иван МГУ ВМК 425  
Октябрь 2021  
UPD: Ноябрь 2021

## Описание
В данном отчете подробно разобраны шаги для создания примитивной поисковой системы, использующей в своей основе векторное представление документов, где значения рассчитываются через tf_idf (двумя способами подсчета tf: count и log(1+count))  

Здесь документы - это предложения из коллекции текстов  
Тексты взяты из прикрепленных ссылок к интересным фактам Википедии  
Запросы - сами формулировку этих фактов (можно увидеть в частях **Запросы** и **Поиск по запросу**)  

При поиске по запросу, сам запрос тоже переводится в векторное пространство документов, и система выдает релевантные документы по мере их близости (близость считается как cos между векторами)  
  
  
### Некоторые ключевые моменты и удобные возможности:
- Даты, римские цифры, английские названия - тоже термы (это может играть роль при поиске документа)  
- Есть возможность сохранять или пересобирать коллекцию с помощью make_collection()  
- Автоматическое добавление новых текстов в коллекцию (нужно записывать новые факты в виде fact_*\<number>*.txt в директорию text и перезапустить сборку коллекции)  
- Обработанная коллекция сохраняется и подгружается как объект pickle (в директории obj)  

### UPD:
Ноябрь 2021
- Реализована также языковая модель с возможностью варьировать влияние коллекции при поиске
- В самом низу есть оценка по NDCG 4 моделей (модель с tf = count и tf = log(1+count), а также язык.модель с lambda = 0.5 или 0.9)


## Необходимые модули

In [1]:
import numpy as np

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

from pymystem3 import Mystem

import re

import os

import pickle

Для обработки русских предложений, может понадобиться установить сначала токенайзер русского языка.

```
import nltk
nltk.download('punkt')
```

## Задача

Запросы – это проанализированные факты из Википедии  

• Коллекция собирается из всех упомянутых статей, из всех фактов  
• Документы – это предложения из статей Википедии, указанных в этих фактах, т.е. коллекция – это объединенная коллекция предложений статей всех фактов  

• Все должно быть обработано морфологическим анализатором  

• Нужно найти наиболее релевантные предложения  
– По tf.idf (df в данном случае – это количество предложений, в которых встречалось слово)  
– Tf –  
    • 1) это количество упоминаний слова в предложении (count) или  
    • 2) log (1+count)  
– Нормализация запроса и предложения  
– Выстроить все предложения из статей по мере сходства с запросом по векторной модели.  
– В отчете должны быть показаны веса выдаваемых предложений  

### + upd за ноябрь

## Запросы

1) Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына  
2) К началу ХХ века на складе казенного чугуноплавильного завода скопился годовой запас продукции  
3) Лагерь сапёров мог стать важнейшим городом Британской Колумбии  

## Решение

### Разбиение текста на предложения

Собираем текст из всех фактов (тексты записаны в файлах **fact_*i*.txt** в исходной директории)

In [2]:
def collect_text():
    reg_file = r'fact_\d.txt'

    file_list = []

    for file_name in sorted(os.listdir('text')):
        if re.match(reg_file, file_name):
            file_list.append('text/'+file_name)

    if file_list == []:
        print("there are no text files")

    all_text = ""

    for file_name in file_list:
        with open(file_name) as file:
            print("open", file_name)
            all_text = all_text + file.read()
    print("done!")

    return all_text

In [3]:
all_text = collect_text()

open text/fact_1.txt
open text/fact_2.txt
open text/fact_3.txt
done!


Разбиваем текст на предложения с помощью токенайзера

In [4]:
def sentence_list(text):
    proc_text = []

    for el in text.split('\n'):
        if el:
            sent_list = sent_tokenize(el, language="russian")
            for s in sent_list:
                proc_text.append(s)
    return proc_text

In [5]:
proc_text = sentence_list(all_text)

In [6]:
proc_text[:3]

['Генрих III (1 октября 1207, Уинчестер — 16 ноября 1272, Вестминстер) — король Англии (1216—1272) и герцог Аквитании из династии Плантагенетов, один из самых малоизвестных британских монархов, при том что правил он дольше всех прочих средневековых королей Англии — 56 лет.',
 'Ранние годы.',
 'Генрих родился 1 октября 1207 года в Уинчестерском замке.']

### Сбор термов, подсчет idf и вероятностей

Регулярное выражение и функция лемматизации:

In [7]:
# кроме русских слов оставим еще даты, а также английские названия,
#   тем самым оставив римские цифры (e.g. III = 3, IV = 4)
reg_filter = r'[а-яА-Я]|[a-zA-Z]|\d'

In [8]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
english_stopwords = stopwords.words("english")

def process_text(text):    
    tokens = mystem.lemmatize(text)
    tokens = [token.lower() 
              for token in tokens 
              if token not in russian_stopwords 
              and token not in english_stopwords
              and token != " "
              and re.match(reg_filter, token) ]
    return tokens

Теперь лемматизируем каждое предложение из **proc_text**, удаляем стоп-слова, проверяем на соответствие регулярному выражению заносим уже термы в словарь термов

In [9]:
terms = {}
term_text = []

for sentence in proc_text:
    new_terms = process_text(sentence)
    term_text.append(new_terms)
    
    for t in new_terms:
        if t not in terms:
            terms[t] = {'df': None, 'idf': None}

Определяем df для каждого терма

In [10]:
for t in terms:
    terms[t]['df'] = 0
    for doc in term_text:
        if t in doc:
            terms[t]['df'] += 1

Считаем idf

In [11]:
number_of_docs = len(proc_text)

In [12]:
for t in terms:
    terms[t]['idf'] = np.log10(number_of_docs / terms[t]['df'])

In [13]:
# посмотрим, что получилось
for i in range(3):
    t = list(terms.keys())[i]
    print(t, terms[t])

генрих {'df': 55, 'idf': 1.0348835702459926}
iii {'df': 24, 'idf': 1.3950350180286304}
1 {'df': 20, 'idf': 1.4742162640762553}


То есть здесь, в конечном итоге, функция выглядит так (**+ upd добавили вероятность**):

In [14]:
def make_terms(proc_text):
    terms = {}
    term_text = []

    for sentence in proc_text:
        new_terms = process_text(sentence)
        term_text.append(new_terms)

        for t in new_terms:
            if t not in terms:
                terms[t] = {'df': None, 'idf': None, 'prob':None}

    # WARN! неэффективный цикл         
    for t in terms:
        terms[t]['df'] = 0
        for doc in term_text:
            if t in doc:
                terms[t]['df'] += 1
                
    # для языковой модели
    all_occurrences_count = 0
    for doc in term_text:
        for t in doc:
            if terms[t]['prob']:
                terms[t]['prob'] += 1
            else:
                terms[t]['prob'] = 1
            all_occurrences_count += 1
    
    number_of_docs = len(proc_text)
    for t in terms:
        terms[t]['idf'] = np.log10(number_of_docs / terms[t]['df'])
        terms[t]['prob'] /= all_occurrences_count
    
    return terms, term_text

In [15]:
terms, term_text = make_terms(proc_text)

### Отображение исходного предложения в вектор пространства термов

Функция нормализации вектора:

In [16]:
def normalize(vec):
    norm = np.linalg.norm(vec)
    return vec/norm

Функция представления вектора в пространстве термов:

In [17]:
def tf_vec(doc, terms):
    words = list(terms.keys())
    doc_vec = np.zeros(len(terms.keys()))

    for t in doc:
        if t in words:
            i = words.index(t)
            doc_vec[i] += 1
        else:
            print("WARN: query word {" + t + "} is not in the collection")
        
    return doc_vec

Функция взвешенного вектора документа (выдает два ответа, соответственно двум разным способам учета tf):

In [18]:
def weight_tf_idf_vec(doc, terms):
    words = terms.keys()
    w_vec = np.zeros(len(terms.keys()))
    
    doc_vec_tf1 = tf_vec(doc, terms)
    doc_vec_tf2 = tf_vec(doc, terms)
    #print("doc = ", doc, "\ndoc_vec after tf_vec():\n", doc_vec_tf1, "\n")
    
    i = 0
    for word in words:
        doc_vec_tf1[i] *= terms[word]['idf']
        doc_vec_tf2[i] = np.log(1+doc_vec_tf2[i]) * terms[word]['idf']
        i += 1
    
    return (doc_vec_tf1, doc_vec_tf2)

На этом моменте у нас есть два списка: исходные предложения **proc_text** и списки термов каждого предложения **term_text**  
  
Для каждого документа из **proc_text** построим векторы по его представлению в **term_text** и запишем их вместе  

In [19]:
proc_collection = list(zip(proc_text, [weight_tf_idf_vec(sent, terms) for sent in term_text]))

Осталось теперь найти релевантные документы из данной коллекции для запроса

Функция сборки коллекции (сохраняет объект pickle):

In [20]:
def make_collection():
    print("building collection...")
    all_text = collect_text()
    proc_text = sentence_list(all_text)
    terms, term_text = make_terms(proc_text)
    proc_collection = list(zip(proc_text, [weight_tf_idf_vec(sent, terms) for sent in term_text]))
    if 'obj' not in os.listdir():
        os.mkdir('obj')
    with open('obj/core_collection.pkl','wb') as f:
        pickle.dump(proc_collection, f, pickle.HIGHEST_PROTOCOL)
    with open('obj/terms.pkl','wb') as f:
        pickle.dump(terms, f, pickle.HIGHEST_PROTOCOL)
    with open('obj/term_text.pkl','wb') as f:
        pickle.dump(term_text, f, pickle.HIGHEST_PROTOCOL)

In [21]:
make_collection()

building collection...
open text/fact_1.txt
open text/fact_2.txt
open text/fact_3.txt
done!


Функция подсчета близости документов:

In [22]:
def similarity(vec1, vec2):
    cos = np.dot(normalize(vec1), normalize(vec2))
    return cos

Функция очистки запроса (неизвестные термы в запросе отбрасываются): **upd языковая модель**

In [23]:
def process_query(query):
    t_query = process_text(query)
    clean_query = []
    
    for t in t_query:
        if t in terms.keys():
            clean_query.append(t)
        else:
            print("WARN: term {" + t + "} not in the collection")
    
    return clean_query

Функция подсчета вероятности P(Q|d): **upd языковая модель**

In [24]:
def prob_query_doc(query, doc, terms, par_lambda):
    P_q_d = 0
    
    for t in query:
        if t in terms.keys():
            if not P_q_d:
                P_q_d = 1

            t_count = len(list(filter(lambda x: x == t, doc)))
            P_q_d *= (1-par_lambda)*terms[t]['prob'] + par_lambda*t_count/len(doc) 
        else:
            print("WARN: term {" + t + "} not in the collection")
    
    return P_q_d

Функция поиска по коллекции с языковой моделью: **upd языковая модель**

In [25]:
def search_lang_model(query, par_lambda):
    obj_name = 'core_collection.pkl'
    if 'obj' not in os.listdir() or obj_name not in os.listdir('obj'):
        make_collection()

    with open('obj/'+obj_name,'rb') as f:
        proc_collection = pickle.load(f)
    with open('obj/terms.pkl','rb') as f:
        terms = pickle.load(f)
    with open('obj/term_text.pkl','rb') as f:
        term_text = pickle.load(f) 
    
    rel_docs = []
    
    c_query = process_query(query)
    
    for i in range(len(proc_collection)):
        rel_docs.append((proc_collection[i][0], prob_query_doc(c_query, 
                                                               term_text[i], terms, par_lambda)))

    rel_docs.sort(key=lambda x:x[1], reverse=True)
    
    return rel_docs

**Функция поиска по коллекции (выдает документы со значениями по мере их релевантности):**

In [26]:
def search(query):
    obj_name = 'core_collection.pkl'
    if 'obj' not in os.listdir() or obj_name not in os.listdir('obj'):
        make_collection()

    with open('obj/'+obj_name,'rb') as f:
        proc_collection = pickle.load(f)
    with open('obj/terms.pkl','rb') as f:
        terms = pickle.load(f)

    vec_q = normalize(tf_vec(process_text(query), terms))

    rel_docs1 = []
    rel_docs2 = []
    
    for i in range(len(proc_collection)):
        vec_d1 = proc_collection[i][1][0] # вектор по подсчету tf = count
        vec_d2 = proc_collection[i][1][1] # вектор по подсчету tf = log(1+count)
        
        rel_docs1.append((proc_collection[i][0], similarity(vec_q, vec_d1)))
        rel_docs2.append((proc_collection[i][0], similarity(vec_q, vec_d2)))
                         
    rel_docs1.sort(key=lambda x:x[1], reverse=True)
    rel_docs2.sort(key=lambda x:x[1], reverse=True)
        
    return rel_docs1, rel_docs2

### Поиск по запросу (tf_idf)

1) Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына  

In [27]:
search('Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына')[0][:3]

WARN: query word {награда} is not in the collection


[('Опека над другой дочерью Випонта, Идонеей, была поручена Роджеру Лейбёрну, женившего на ней своего сына.',
  0.3497588024421523),
 ('Кроме того, Клиффорду была поручена опека над Изабеллой, одной из дочерей мятежного барона Роберта де Випонта, на которой он женил своего наследника.',
  0.3080062523852433),
 ('На ближайшие семь лет опекунами короля были назначены сторонники Дорварда, причем сместить их мог только король Англии.',
  0.2687454450045788)]

In [28]:
search('Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына')[1][:3]

WARN: query word {награда} is not in the collection


[('Опека над другой дочерью Випонта, Идонеей, была поручена Роджеру Лейбёрну, женившего на ней своего сына.',
  0.3497588024421523),
 ('Кроме того, Клиффорду была поручена опека над Изабеллой, одной из дочерей мятежного барона Роберта де Випонта, на которой он женил своего наследника.',
  0.3080062523852433),
 ('На ближайшие семь лет опекунами короля были назначены сторонники Дорварда, причем сместить их мог только король Англии.',
  0.2545444967241659)]

2) К началу ХХ века на складе казенного чугуноплавильного завода скопился годовой запас продукции  

In [29]:
search('К началу XX века на складе казенного чугуноплавильного завода скопился годовой запас продукции')[0][:10]

WARN: query word {казенный} is not in the collection
WARN: query word {скопиться} is not in the collection
WARN: query word {запас} is not in the collection


[('XX век.', 0.49622824011683675),
 ('Экономический кризис начала XX века почти не сказался на работе Баранчинского завода, работавшего по государственным заказам.',
  0.36536559395037094),
 ('По инициативе Шувалова Баранчинский завод был реконструирован в чугуноплавильный.',
  0.24106812342337336),
 ('Но из-за отсутствия сторонних заказов на 1 января 1904 года на складах завода накопилось 698 тыс. пудов товарного чугуна, что превышало его годовую выплавку.',
  0.23425064334158735),
 ('XIX век.', 0.2174647574608576),
 ('Роджер был похоронен в аббатстве Дор в Херефордшире.Баранчинский (Нижне-Баранчинский, Баранчинский Нижний) чугуноплавильный и железоделательный завод — металлургический завод на Среднем Урале, основанный в середине XVIII века.',
  0.21033327984950198),
 ('Баранчинский завод имел логистическое преимущество перед другими металлургическими заводами округа из-за относительной близости (60 вёрст) к Ослянской пристани на реке Чусовой, по которой осуществлялась доставка продук

In [30]:
search('К началу ХХ века на складе казенного чугуноплавильного завода скопился годовой запас продукции')[1][:3]

WARN: query word {хх} is not in the collection
WARN: query word {казенный} is not in the collection
WARN: query word {скопиться} is not in the collection
WARN: query word {запас} is not in the collection


[('Экономический кризис начала XX века почти не сказался на работе Баранчинского завода, работавшего по государственным заказам.',
  0.25827223250024517),
 ('По инициативе Шувалова Баранчинский завод был реконструирован в чугуноплавильный.',
  0.2577126642065135),
 ('Но из-за отсутствия сторонних заказов на 1 января 1904 года на складах завода накопилось 698 тыс. пудов товарного чугуна, что превышало его годовую выплавку.',
  0.2504244714330281)]

3) Лагерь сапёров мог стать важнейшим городом Британской Колумбии  

In [31]:
search('Лагерь сапёров мог стать важнейшим городом Британской Колумбии')[0][:3]

[('Название провинции было выбрано королевой Викторией, когда колония Британской Колумбии стала британской в 1858 году.',
  0.2622196898521864),
 ('Столица провинции, город Виктория с населением 85 792 человек не входит в число 10 крупнейших городов Британской Колумбии.',
  0.23909099746446222),
 ('Туризм также стали играть важную роль в экономике.', 0.22778274362385423)]

In [32]:
search('Лагерь сапёров мог стать важнейшим городом Британской Колумбии')[1][:3]

[('Название провинции было выбрано королевой Викторией, когда колония Британской Колумбии стала британской в 1858 году.',
  0.2423808893362787),
 ('Туризм также стали играть важную роль в экономике.', 0.2277827436238542),
 ('Столица провинции, город Виктория с населением 85 792 человек не входит в число 10 крупнейших городов Британской Колумбии.',
  0.21712823838191134)]

### Поиск по запросу (языковая модель)
upd Ноябрь 2021

In [33]:
search_lang_model('Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына', 0.5)[:3]

WARN: term {награда} not in the collection


[('Опека над другой дочерью Випонта, Идонеей, была поручена Роджеру Лейбёрну, женившего на ней своего сына.',
  1.5766756735762248e-25),
 ('Кроме того, Клиффорду была поручена опека над Изабеллой, одной из дочерей мятежного барона Роберта де Випонта, на которой он женил своего наследника.',
  2.838362701495031e-26),
 ('На ближайшие семь лет опекунами короля были назначены сторонники Дорварда, причем сместить их мог только король Англии.',
  1.5164850929571055e-27)]

In [34]:
search_lang_model('Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына', 0.9)[:3]

WARN: term {награда} not in the collection


[('Опека над другой дочерью Випонта, Идонеей, была поручена Роджеру Лейбёрну, женившего на ней своего сына.',
  9.782270298915218e-29),
 ('Кроме того, Клиффорду была поручена опека над Изабеллой, одной из дочерей мятежного барона Роберта де Випонта, на которой он женил своего наследника.',
  1.6844351824499435e-29),
 ('На ближайшие семь лет опекунами короля были назначены сторонники Дорварда, причем сместить их мог только король Англии.',
  1.0496661228404642e-31)]

In [35]:
search_lang_model('К началу ХХ века на складе казенного чугуноплавильного завода скопился годовой запас продукции', 0.5)[:3]

WARN: term {хх} not in the collection
WARN: term {казенный} not in the collection
WARN: term {скопиться} not in the collection
WARN: term {запас} not in the collection


[('Но из-за отсутствия сторонних заказов на 1 января 1904 года на складах завода накопилось 698 тыс. пудов товарного чугуна, что превышало его годовую выплавку.',
  1.3291276525000708e-19),
 ('Роджер был похоронен в аббатстве Дор в Херефордшире.Баранчинский (Нижне-Баранчинский, Баранчинский Нижний) чугуноплавильный и железоделательный завод — металлургический завод на Среднем Урале, основанный в середине XVIII века.',
  4.105150116760321e-20),
 ('Экономический кризис начала XX века почти не сказался на работе Баранчинского завода, работавшего по государственным заказам.',
  1.594354088187172e-20)]

In [36]:
search_lang_model('К началу ХХ века на складе казенного чугуноплавильного завода скопился годовой запас продукции', 0.9)[:3]

WARN: term {хх} not in the collection
WARN: term {казенный} not in the collection
WARN: term {скопиться} not in the collection
WARN: term {запас} not in the collection


[('Но из-за отсутствия сторонних заказов на 1 января 1904 года на складах завода накопилось 698 тыс. пудов товарного чугуна, что превышало его годовую выплавку.',
  1.1341120752875626e-21),
 ('Роджер был похоронен в аббатстве Дор в Херефордшире.Баранчинский (Нижне-Баранчинский, Баранчинский Нижний) чугуноплавильный и железоделательный завод — металлургический завод на Среднем Урале, основанный в середине XVIII века.',
  3.587906757079279e-22),
 ('Экономический кризис начала XX века почти не сказался на работе Баранчинского завода, работавшего по государственным заказам.',
  1.3757670068981924e-22)]

In [37]:
search_lang_model('Лагерь сапёров мог стать важнейшим городом Британской Колумбии', 0.5)[:3]

[('Только с согласия этого органа король мог назначать министров и принимать важные решения по управлению страной.',
  1.938026102328954e-21),
 ('Туризм также стали играть важную роль в экономике.',
  1.1109964792433904e-21),
 ('Название провинции было выбрано королевой Викторией, когда колония Британской Колумбии стала британской в 1858 году.',
  8.273622232318016e-22)]

In [38]:
search_lang_model('Лагерь сапёров мог стать важнейшим городом Британской Колумбии', 0.9)[:3]

[('Название провинции было выбрано королевой Викторией, когда колония Британской Колумбии стала британской в 1858 году.',
  1.3285590341311861e-24),
 ('Столица Британской Колумбии — город Виктория — расположена в юго-восточной оконечности острова Ванкувер.',
  1.2964515386256877e-24),
 ('Столица провинции, город Виктория с населением 85 792 человек не входит в число 10 крупнейших городов Британской Колумбии.',
  7.735385586740874e-25)]

## Оценка  моделей по NDCG:

Отдельно для каждого запроса разметим документы по релевантности от 0 до 4

Подготовим словарь, в котором релевантность каждого документа будет равна 0, и уже после отметим релевантные документы по нашему выбору

In [39]:
rel_docs = dict.fromkeys(proc_text, 0)

In [40]:
# переинициализируем словарь, все значения на 0
def reinit_rel_docs(rel_docs):
    for d in rel_docs:
        rel_docs[d] = 0

In [41]:
# оставляем документы, выбрасываем значения близости к запросу
# под документы делаем список в соответствие с экспертной оценкой
def get_rel_sys_ans(sys_docs_tuples):
    sys_docs = [d[0] for d in sys_docs_tuples]
    return [rel_docs[d] for d in sys_docs]

In [42]:
# вычисляем списки релевантности выданных документов от систем
def sys_lists(query):
    sys1 = get_rel_sys_ans( search(query)[0] )
    sys2 = get_rel_sys_ans( search(query)[1] )
    sys3 = get_rel_sys_ans( search_lang_model(query, 0.5) )
    sys4 = get_rel_sys_ans( search_lang_model(query, 0.9) )
    return (sys1, sys2, sys3, sys4)

In [43]:
def DCG(rel_sys):
    dcg = 0
    
    count = 1
    for rel in rel_sys:
        dcg += rel / np.log2(count+1)
        count += 1
        
    return dcg

In [44]:
def better_ordering(rel_sys):
    return sorted(rel_sys, reverse=True)

In [45]:
def NDCG(rel_sys):
    dcg = DCG(rel_sys)
    
    irel_sys = better_ordering(rel_sys)
    
    #print(rel_sys[:4])
    #print(irel_sys[:4])
    
    idcg = DCG(irel_sys)
        
    return dcg / idcg

In [46]:
# вывод результата
def print_ndcg_for_systems(query):
    print('Оценка моделей по запросу:')
    print('\t', query, '\n')
    print('tf_idf (tf = count) \t\t', NDCG(sys2))
    print('tf_idf (tf = log(1+ count)) \t', NDCG(sys2))
    print('Язык.модель (лямбда 0.5) \t\t', NDCG(sys3))
    print('Язык.модель (лямбда 0.9) \t\t', NDCG(sys4))

### Запрос 1

In [47]:
query1 = 'Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына'

In [48]:
reinit_rel_docs(rel_docs)

In [49]:
rel_docs['Кроме того, Клиффорду была поручена опека над Изабеллой, одной из дочерей мятежного барона Роберта де Випонта, ' + 
         'на которой он женил своего наследника.'] = 4
rel_docs['Опека над другой дочерью Випонта, Идонеей, была поручена Роджеру Лейбёрну, женившего на ней своего сына.'] = 2

In [50]:
sys1, sys2, sys3, sys4 = sys_lists(query1)

WARN: query word {награда} is not in the collection
WARN: query word {награда} is not in the collection
WARN: term {награда} not in the collection
WARN: term {награда} not in the collection


In [51]:
print_ndcg_for_systems(query1)

Оценка моделей по запросу:
	 Верный королю барон в награду был назначен опекуном дочери мятежника и женил на ней своего сына 

tf_idf (tf = count) 		 0.8597186998521972
tf_idf (tf = log(1+ count)) 	 0.8597186998521972
Язык.модель (лямбда 0.5) 		 0.8597186998521972
Язык.модель (лямбда 0.9) 		 0.8597186998521972


### Запрос 2

In [52]:
query2 = 'К началу XX века на складе казенного чугуноплавильного завода скопился годовой запас продукции'

In [53]:
reinit_rel_docs(rel_docs)

In [54]:
rel_docs['Экономический кризис начала XX века почти не сказался на работе Баранчинского завода, работавшего по государственным заказам.'] = 1
rel_docs['Но из-за отсутствия сторонних заказов на 1 января 1904 года на складах завода накопилось 698 тыс. пудов товарного чугуна, что превышало его годовую выплавку.'] = 3

In [55]:
sys1, sys2, sys3, sys4 = sys_lists(query2)

WARN: query word {казенный} is not in the collection
WARN: query word {скопиться} is not in the collection
WARN: query word {запас} is not in the collection
WARN: query word {казенный} is not in the collection
WARN: query word {скопиться} is not in the collection
WARN: query word {запас} is not in the collection
WARN: term {казенный} not in the collection
WARN: term {скопиться} not in the collection
WARN: term {запас} not in the collection
WARN: term {казенный} not in the collection
WARN: term {скопиться} not in the collection
WARN: term {запас} not in the collection


In [56]:
print_ndcg_for_systems(query2)

Оценка моделей по запросу:
	 К началу XX века на складе казенного чугуноплавильного завода скопился годовой запас продукции 

tf_idf (tf = count) 		 0.5296052411645183
tf_idf (tf = log(1+ count)) 	 0.5296052411645183
Язык.модель (лямбда 0.5) 		 0.6885288809404666
Язык.модель (лямбда 0.9) 		 0.7967075809905066


### Запрос 3

In [57]:
query3 = 'Лагерь сапёров мог стать важнейшим городом Британской Колумбии'

In [58]:
reinit_rel_docs(rel_docs)

In [59]:
rel_docs['Название провинции было выбрано королевой Викторией, когда колония Британской Колумбии стала британской в 1858 году.'] = 1
rel_docs['Столица провинции, город Виктория с населением 85 792 человек не входит в число 10 крупнейших городов Британской Колумбии.'] = 1
rel_docs['Королева Виктория выбрала название Британская Колумбия, чтобы отличать её от округа Колумбия в Соединённых Штатах («Американская Колумбия» или «Южная Колумбия»), которая стала территорией штата Орегон в 1848 году в результате договора.'] = 2
rel_docs['Привлечение рабочей силы для развития провинции было проблематичным с самого начала, и Британская Колумбия стала местом иммиграции из Европы, Китая и Японии.'] = 1
rel_docs['Туризм также стали играть важную роль в экономике.'] = 1
rel_docs['Порт-Муди — город в провинции Британская Колумбия (Канада), часть агломерации Метро-Ванкувер, центр металлургической, химической и нефтеперерабатывающей промышленности, глубоководный грузовой порт.'] = 3

In [60]:
sys1, sys2, sys3, sys4 = sys_lists(query3)

In [61]:
print_ndcg_for_systems(query3)

Оценка моделей по запросу:
	 Лагерь сапёров мог стать важнейшим городом Британской Колумбии 

tf_idf (tf = count) 		 0.6706616692282683
tf_idf (tf = log(1+ count)) 	 0.6706616692282683
Язык.модель (лямбда 0.5) 		 0.5516894924773739
Язык.модель (лямбда 0.9) 		 0.6149940092691097
